# Topic Modeling with LDA
Use this notebook to practice topic modeling (see lecture slides for reference). First, the usual stuff.

In [1]:
import pandas as pd
transcript_df = pd.read_excel('data/excel/edi_2024_daniel_george.xlsx')
transcript_df.sample(3)

,Unnamed: 0,timestamp,speaker,utterance
441,441,00:38:52,Daniel,And. Make a cool music.
116,116,00:09:35,George,I think that's nice.
10,10,00:00:29,George,Yeah. okay uhh..


In [2]:
utterances = transcript_df['utterance'].tolist()
utterances[:3]

['Just record the voice and then next we can do something...',
 'Yes.',
 'But we need to make a video as well.']

In [3]:
def remove_puncts(utterance_text, alphanumeric_only='True'):
    utterance_text = utterance_text.replace('-', ' ')
    clean_utterance_text = ''.join(e for e in utterance_text if e.isalnum() or e == ' ').lower()
    clean_utterance_text = ' '.join(clean_utterance_text.split())
    return clean_utterance_text

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def get_words_tokenized_nopunct_nostop(utterances, stop_w=stop_words):
    utterance_words_list = []
    for utterance in utterances:
        clean_utterance = remove_puncts(utterance)
        words = word_tokenize(clean_utterance.lower())
        words_nostop = [word for word in words if not word in stop_w]
        utterance_words_list.append(words_nostop)
    return utterance_words_list

In [5]:
tokenized_utterances_list = get_words_tokenized_nopunct_nostop(utterances)

## Train an LDA Model
LDA models require an estimated number of topics. We can choose this to be 5, and fine-tune if we see overlaps.

In [6]:
import gensim

dictionary = gensim.corpora.Dictionary(tokenized_utterances_list)
corpus = [dictionary.doc2bow(utterance) for utterance in tokenized_utterances_list]
topic_count = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topic_count, id2word=dictionary, passes=50)

## Visualize the LDA Model
We use the famouse [pyLDAvis](https://pypi.org/project/pyLDAvis/) library to make sense of the topics.

First, we set up the notebook to suppress warnings (you'll get a lot due to package deprecations).

In [7]:
import warnings
warnings.filterwarnings('ignore')

## IMPORTANT: Set the slider below to a $\lambda$ value of 0.4
A $\lambda$ of 1 shows all frequently-occurring words, which gives you a global picture of the text in general, but does not help you understand the topics. A $\lambda$ of 0.1 shows only the words unique to each topic, which helps you understand the difference between the topics, but not what the topics are about. A $\lambda$ of 0.4 achieves a good balance. See [this paper](https://aclanthology.org/W14-3110.pdf) for more details.

In [8]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
gensimvis.prepare(ldamodel, corpus, dictionary)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=91619) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <numbe

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.005062 -0.035435       1        1  22.937002
3      0.235381  0.045299       2        1  21.209231
1     -0.141266  0.143656       3        1  20.815930
2     -0.057076 -0.180987       4        1  19.744791
0     -0.031977  0.027466       5        1  15.293045, topic_info=      Term       Freq      Total Category  logprob  loglift
9     yeah  89.000000  89.000000  Default  30.0000  30.0000
29      ok  34.000000  34.000000  Default  29.0000  29.0000
10     one  26.000000  26.000000  Default  28.0000  28.0000
34   right  16.000000  16.000000  Default  27.0000  27.0000
78   think  21.000000  21.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
5     make   4.306318  35.435737   Topic5  -3.8105  -0.2299
21    cool   1.739511   7.856928   Topic5  -4.7170   0.3700
38    like   2.988991  36.864233   Topic5  -4.1756  -0.6345
224  audio   0.951046   3.143741   Topic5  -5.3208   0.6822
234      2   0.950519   2.354997   Topic5  -5.3213   0.9705

[253 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
272       1  0.260559    10
272       2  0.521118    10
234       4  0.424629     2
234       5  0.424629     2
266       4  0.956581    20
...     ...       ...   ...
9         1  0.089704  yeah
9         2  0.908254  yeah
4         1  0.217711   yes
4         2  0.544276   yes
4         3  0.217711   yes

[296 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 3, 1])

## View topic distribution for a document (review)

In [9]:
ldamodel.get_document_topics(corpus[0])

[(0, 0.04194761),
 (1, 0.0402997),
 (2, 0.04059116),
 (3, 0.8371535),
 (4, 0.040008053)]

# Exercise: What are some topics relevant to you?
Explore the topics and their keywords. 